In [38]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn import cross_validation
#from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time 
from sklearn import neighbors
from sklearn.utils import shuffle
tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [39]:
mnist = fetch_mldata("MNIST original")

In [40]:
print(mnist.data.shape)

(70000, 784)


In [41]:
print(np.unique(mnist.target))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [42]:
X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X, y = shuffle(X,y)

In [43]:
#X, y = np.float32(mnist.data[:70000]), np.float32(mnist.target[:70000])
X_train, y_train = np.float32(X[:15000]), np.float32(y[:15000])
X_test, y_test = np.float32(X[60000:]), np.float32(y[60000:])

# PCA 

In [44]:
pca=PCA(n_components=700)
X_train_pca=pca.fit_transform(X_train,y_train)
X_test_pca=pca.transform(X_test)
print('Variance covered', pca.explained_variance_ratio_.sum())
print('PCA Shape', X_train_pca.shape)

Variance covered 1.0
PCA Shape (15000, 700)


# LDA

In [45]:
lda = LinearDiscriminantAnalysis()
X_train_lda=lda.fit_transform(X_train_pca,y_train)
X_test_lda=lda.transform(X_test_pca)
print('Variance covered', lda.explained_variance_ratio_.sum())
print('LDA Shape', X_train_lda.shape)

Variance covered 1.0
LDA Shape (15000, 9)


In [46]:
start = int(round(time.time() * 1000))

In [47]:
DT=tree.DecisionTreeClassifier(random_state=1)
DT.fit(X_train_lda,y_train)
predicted=DT.predict(X_test_lda)
expected=y_test

In [48]:
print(cross_validation.cross_val_score(DT, X_train_lda,y_train, cv=5))

[ 0.83926789  0.84705098  0.84766667  0.84022682  0.83845127]


In [49]:
end = int(round(time.time() * 1000))
print("--NN fitting finished in ", (end-start), "ms--------------")

--NN fitting finished in  2097 ms--------------


In [50]:
print("Classification report for kNN classifier %s:\n%s\n"
     % (DT, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Classification report for kNN classifier DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best'):
             precision    recall  f1-score   support

        0.0       0.89      0.91      0.90       964
        1.0       0.90      0.90      0.90      1127
        2.0       0.77      0.79      0.78      1014
        3.0       0.77      0.75      0.76      1040
        4.0       0.81      0.84      0.82       960
        5.0       0.73      0.71      0.72       901
        6.0       0.90      0.86      0.88      1008
        7.0       0.84      0.84      0.84      1092
        8.0       0.71      0.70      0.70       938
        9.0       0.74      0.78      0.76       956

avg / total       0.81    

In [51]:
print("Accuracy is:",round(metrics.accuracy_score(expected,predicted)*100,2))

Accuracy is: 80.98


In [52]:
print("Test error is:",100-round(metrics.accuracy_score(expected,predicted)*100,2))

Test error is: 19.02


In [53]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

Confusion matrix:
[[ 877    2   17    2    3   20   16    2   13   12]
 [   1 1011   24   10    8    4    1   25   38    5]
 [  17   25  804   45   18   19   24   19   21   22]
 [   5   10   50  778    6   72    4   28   59   28]
 [   1    7   16    3  802   13   17   18   11   72]
 [  22    7   24   67   11  644   14    9   77   26]
 [  23    8   26   10   23   28  864    1   22    3]
 [  10    8   20   30   19    9    2  913    9   72]
 [  13   44   51   45   24   56   18   13  655   19]
 [  12    2    8   18   77   16    1   55   17  750]]
